# Get Kaggle IEEE Fraud Data

In [84]:
!pip install kaggle

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
  Using cached urllib3-1.24.3-py2.py3-none-any.whl (118 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.25.7
    Uninstalling urllib3-1.25.7:
      Successfully uninstalled urllib3-1.25.7


In [ ]:
!kaggle competitions download -c ieee-fraud-detection

 63%|████████████████████████▍              | 74.0M/118M [00:15<00:09, 4.90MB/s]

# SAVE TO GOOGLE DRIVE

In [67]:
from catboost import CatBoostClassifier, Pool
from catboost.utils import get_roc_curve
from sklearn.metrics import auc, roc_auc_score
import tqdm
import sys
import os
sys.path.append(os.path.abspath('../src'))
import data_prep, ai_config
import importlib
importlib.reload(data_prep)
importlib.reload(ai_config)

<module 'ai_config' from '/Users/wihill/git/AI_Based_Hyperparameter_Tuning/src/ai_config.py'>

In [83]:
for i in range(0,3):

    print('\nfold',i)

    test, train = data_prep.get_fold(i, '../folds')

    model = CatBoostClassifier(iterations=1,
                               depth=16,
                               learning_rate=1,
                               custom_metric=['AUC'],
                               eval_metric='AUC',
                               verbose=False)

    catboost_pool = Pool(train.drop([ai_config.Y_COL], axis=1), train[[ai_config.Y_COL]])
    model.fit(catboost_pool)
    (fpr, tpr, thresholds) = get_roc_curve(model, catboost_pool, plot=False)
    print('\ntrain auc',auc(fpr, tpr))
    print('test auc',roc_auc_score(test[[ai_config.Y_COL]], model.predict(test.drop([ai_config.Y_COL], axis=1))))


fold 0
train auc 0.9855086571827787
test auc 0.7703881160569014

fold 1


KeyboardInterrupt: 

In [4]:
cat_features = [0]

cv_dataset = Pool(data=cv_data,
                  label=labels,
                  cat_features=cat_features)

params = {"iterations": 100,
          "depth": 2,
          "loss_function": "Logloss",
          "verbose": False}

scores = cv(cv_dataset,
            params,
            fold_count=2, 
            plot="True")